In [34]:
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Dog-or-Cat-or-Panda'

In [35]:
def load_data():
    labels = {}
    labels_r = {}
    data = []
    idx = -1
    for folder in tqdm(os.listdir('./data/')):
        idx += 1
        labels[folder] = idx
        labels_r[idx] = folder
        for file in os.listdir(f'./data/{folder}'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(112,112))
            img = img / 255.0
            data.append([img,idx])
    np.random.shuffle(data)
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=False,test_size=0.0625)
    return X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data

In [36]:
X_train,X_test,y_train,y_test,X,y,labels,labels_r,idx,data = load_data()

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.63it/s]


In [37]:
labels

{'cats': 0, 'panda': 1}

In [38]:
X_train = torch.from_numpy(np.array(X_train)).view(-1,3,112,112).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).view(-1,3,112,112).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [39]:
def get_loss(model,X,y,criterion):
    return criterion(model(X).view(-1,1),y.view(-1,1)).item()

In [40]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [41]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.max_pool2d = MaxPool2d((2,2),(2,2))
        self.activation = ReLU()
        self.conv1 = Conv2d(3,7,(5,5))
        self.conv1bn = BatchNorm2d(7)
        self.conv2 = Conv2d(7,14,(5,5))
        self.conv2bn = BatchNorm2d(14)
        self.conv3 = Conv2d(14,21,(5,5))
        self.conv3bn = BatchNorm2d(21)
        self.linear1 = Linear(21*10*10,256)
        self.linear1bn = BatchNorm1d(256)
        self.linear2 = Linear(256,512)
        self.linear2bn = BatchNorm1d(512)
        self.linear3 = Linear(512,256)
        self.linear3bn = BatchNorm1d(256)
        self.output = Linear(256,1)
        self.output_ac = Sigmoid()
    
    def forward(self,X):
        preds = self.max_pool2d(self.activation(self.conv1bn(self.conv1(X))))
        preds = self.max_pool2d(self.activation(self.conv2bn(self.conv2(preds))))
        preds = self.max_pool2d(self.activation(self.conv3bn(self.conv3(preds))))
#         print(preds.shape)
        preds = preds.view(-1,21*10*10)
        preds = self.activation(self.linear1bn(self.linear1(preds)))
        preds = self.activation(self.linear2bn(self.linear2(preds)))
        preds = self.activation(self.linear3bn(self.linear3(preds)))
        preds = self.output_ac(self.output(preds))
        return preds

In [42]:
model = Model().to(device)

In [43]:
criterion = BCELoss()

In [44]:
from torch.optim import *

In [45]:
optimizer = Adam(model.parameters(),lr=0.001)

In [46]:
epochs = 100

In [47]:
batch_size = 32

In [48]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size].to(device).float().view(-1,3,112,112)
        y_batch = y_train[i:i+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion))/2})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    wandb.log({'Val ACC':get_accuracy(model,X_test,y_test)})
wandb.watch(model)
wandb.finish()

100%|█████████████████████████████████████████| 100/100 [00:15<00:00,  6.27it/s]


Loss,0.00019
_runtime,22
_timestamp,1631183047
_step,399
Val Loss,0.43542
Acc,100.0
Val ACC,93.5


Loss,█▄▃▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Val Loss,▄▂▁▃▂▂▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
Acc,▁▅▇█████████████████████████████████████
Val ACC,▁██▁██▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
